In [1]:
! pip install rouge_score
! pip install -U accelerate


In [2]:
!pip install transformers datasets

In [3]:
#!pip install bitsandbytes

In [4]:
!pip install wandb

In [5]:
import glob
import wandb
import matplotlib.pyplot as plt
# import csv
from io import StringIO
from transformers import pipeline, set_seed

import matplotlib.pyplot as plt
from datasets import load_metric#, load_from_disk

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch
from datasets import Dataset
from sklearn.metrics import f1_score
import numpy as np
import os

# nltk.download("punkt")

In [6]:
import accelerate

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
os.environ["WANDB_PROJECT"] = "CompactBioBert" # name your W&B project
os.environ["WANDB_LOG_MODEL"] = "checkpoint" # log all model checkpoints

In [9]:
# folder_path = '/content/drive/MyDrive/train'
# file_list = glob.glob(folder_path + "/*.txt")

In [10]:
#df = pd.DataFrame(pd.read_table(file_list[0],on_bad_lines='skip',quoting=csv.QUOTE_NONE,names = ["Text"],chunksize = 1000000))
# #df = pd.DataFrame(pd.read_table(file_list[0],names = ["Text"],delimiter = '\t'))
def read_paper(path):
  f = open(path, 'r', encoding="utf-8")
  text = str(f.read())
  f.close()
  return text

In [11]:
# text = read_paper('/content/drive/MyDrive/train/189471.txt')

In [12]:
def create_list(folder_path):
  temp_papers = []
  #filenames = []
  for filename in tqdm(glob.glob(folder_path + "/*.txt")):
      temp_papers.append(read_paper(filename))
     # filenames.append(filename)
  for filename in tqdm(glob.glob(folder_path + "/*.xml")):
      temp_papers.append(read_paper(filename))
     # filenames.append(filename)
 # return pd.DataFrame(list(zip(temp_papers, filenames)), columns =['text', 'filenames'])
  return temp_papers

In [13]:
train_path = '/content/drive/MyDrive/train'
val_path = '/content/drive/MyDrive/validation'
test_path = '/content/drive/MyDrive/test'

In [14]:
#df = pd.read_csv(StringIO(text),on_bad_lines='skip',quoting=csv.QUOTE_NONE,names = ["Text"])
list_train = create_list(train_path)
list_val = create_list(val_path)
list_test = create_list(test_path)

100%|██████████| 86/86 [00:00<00:00, 1018.61it/s]


In [15]:
list_train[1]

"Admission Date:  [**2133-3-28**]       Discharge Date:  [**2133-4-2**]\n\nDate of Birth:   [**2072-12-20**]       Sex:  M\n\nService:  [**Company 191**]\n\nHISTORY OF PRESENT ILLNESS:  This is a 60-year-old man with\nCAD, PVD who had difficulty speaking after left carotid\nendarterectomy in [**2133-2-16**].  The patient has had bilateral\ncarotid stenosis.  On [**2-21**] while he was sitting in a chair he\ndeveloped sudden onset of right arm and leg numbness,\nfollowed by right arm and leg weakness.  He also had\ndifficulty speaking.  He was admitted to [**Hospital3 **]\nand underwent a left carotid endarterectomy on [**2-26**] and\nafterwards began having severe left sided headache behind his\nleft eye that lasted for hours and was constant.\nNevertheless, he visited [**Hospital3 **] for continued\nheadaches and nausea and vomiting.  During one of those\nvisits he had a contortion of his right face and bilateral\narm jerking and was started on Dilantin with presumptive\ndiagnosis of 

In [16]:
list_train[-1]

'<?xml version="1.0" encoding="UTF-8" ?>\n<PatientMatching>\n<TEXT><![CDATA[\n\nRecord date: 2097-06-09\n\n\n \n\n\nRenal Consult Note\nQuinton Delong,M.D.\n\n\n\n\nCC/ID: Asked to assit in the mangement of Lori Stevens with hypertension, nausea and vomiting.\n\n\n\n\nHISTORY OF PRESENT ILLNESS:\n\n\n\nShe is a 62 year old lady with ESRD who has been admitted in the past with nausea, vomiting and severe hypertension. She was treated with BP medications. She has had a CT scan and is seen in dialysis. She is stable on dialysis currently.\n\n\n\n\n\n\nPAST MEDICAL HISTORY:\n\n1.       Diabetes Mellitus x many years.\n2.       History of hypertension x many years.\n3.       History of GI bleeding.\n4.       ESRD secondary to DM on HD x years. On HD Mon/Wed/Fri.\n5.       CAD s/p NSTEMI.\n6.       Meningitis in the past. ( Viral)\n      \n\n\nMEDICATIONS\n\nUnavailable\n\n\nSOCIAL HISTORY:\n\n\nLives in Nursing Home.\n\n\n\n\nPHYSICAL EXAMINATION:\n\n\n\n\nBP: 159/70  P:  72   R:  16    T=9

In [17]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [18]:
model = "nlpie/compact-biobert"

tokenizer = AutoTokenizer.from_pretrained(model)

model_compactbiobert = AutoModelForMaskedLM.from_pretrained(model).to(device)

In [19]:
# def convert_examples_to_features(example_batch):
#     input_encodings = tokenizer(example_batch['text'] , max_length = 1024, truncation = True )

#     return {
#         'input_ids' : input_encodings['input_ids'],
#         'attention_mask': input_encodings['attention_mask'],
#     }

#df_train_1 = df_train.map(convert_examples_to_features, batched = True)
# df_test_1 = df_test.applymapmap(convert_examples_to_features)

#df_train["tokens"] = df_train["text"].apply(tokenizer.tokenize(truncation=True, max_length=512))

#df_train.drop(["tokens"],axis=1,inplace=True)

encoded_data_train = tokenizer.batch_encode_plus(
    list_train,
    add_special_tokens=True,
    return_attention_mask=True,
    truncation=True,
    padding='longest',
    max_length=512,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    list_val,
    add_special_tokens=True,
    return_attention_mask=True,
    truncation=True,
    padding='longest',
    max_length=512,
    return_tensors='pt'
)
# input_ids_train = encoded_data_train['input_ids']
# attention_masks_train = encoded_data_train['attention_mask']
# tags_train = torch.tensor(df[df.data_type=='train']['Tags Mapped'].values)

In [20]:
type(encoded_data_val)

transformers.tokenization_utils_base.BatchEncoding

In [21]:
print(tokenizer)

BertTokenizerFast(name_or_path='nlpie/compact-biobert', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


In [22]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15, return_tensors="pt")

In [23]:
savePath = "/content/drive/MyDrive/models/"

In [24]:
trainingArguments = TrainingArguments(
    savePath + "checkpoints",
    logging_steps=1,
    overwrite_output_dir=True,
    #save_steps=2500,
    num_train_epochs=100,
    learning_rate=5e-5,
    lr_scheduler_type="linear",
    #evaluate_during_training=True,
    do_train=True,
    do_eval=True,
    warmup_steps=5000,
    per_device_train_batch_size=32, #In our experiments 4 GPUs were used
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=128,
    metric_for_best_model='eval_loss',
    weight_decay=1e-4,
    fp16=True,
    report_to="wandb",
    gradient_checkpointing=True,
    optim="adamw_torch",
    save_total_limit=2,
    remove_unused_columns=True,
    evaluation_strategy = 'steps',
    eval_steps = 1,
    save_strategy = "steps",
    load_best_model_at_end=True
)

In [25]:
# for using bitsandbinary
# import bitsandbytes as bnb
# from torch import nn
# from transformers.trainer_pt_utils import get_parameter_names

# decay_parameters = get_parameter_names(model_compactbiobert, [nn.LayerNorm])
# decay_parameters = [name for name in decay_parameters if "bias" not in name]
# optimizer_grouped_parameters = [
#     {
#         "params": [p for n, p in model_compactbiobert.named_parameters() if n in decay_parameters],
#         "weight_decay": trainingArguments.weight_decay,
#     },
#     {
#         "params": [p for n, p in model_compactbiobert.named_parameters() if n not in decay_parameters],
#         "weight_decay": 0.0,
#     },
# ]

# optimizer_kwargs = {
#     "betas": (trainingArguments.adam_beta1, trainingArguments.adam_beta2),
#     "eps": trainingArguments.adam_epsilon,
# }
# optimizer_kwargs["lr"] = trainingArguments.learning_rate
# adam_bnb_optim = bnb.optim.Adam8bit(
#     optimizer_grouped_parameters,
#     betas=(trainingArguments.adam_beta1, trainingArguments.adam_beta2),
#     eps=trainingArguments.adam_epsilon,
#     lr=trainingArguments.learning_rate,
# )

In [26]:
dataset_train = Dataset.from_dict(encoded_data_train)
dataset_val = Dataset.from_dict(encoded_data_val)

In [27]:
dataset_val

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 103
})

In [28]:
trainer = Trainer(model=model_compactbiobert, args=trainingArguments,
                  tokenizer=tokenizer, data_collator=data_collator,
                  train_dataset=dataset_train,eval_dataset=dataset_val)#,optimizers=(adam_bnb_optim, None))

In [29]:
trainer.train()

wandb: Currently logged in as: gravisandeep (ravigorti). Use `wandb login --relogin` to force relogin


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
1,0.779500,4.563742
2,0.784500,4.635225
3,0.778500,4.574714
4,0.783600,4.634483
5,0.781400,4.545599
6,0.776300,4.556579
7,0.075000,4.624746
8,0.708600,4.545418
9,0.779100,4.562931
10,0.778400,4.479971


KeyboardInterrupt: ignored

In [ ]:
trainer.save_model(savePath + "Compact_BioBERT/")

In [ ]:
# model_compactbiobert = AutoModelForMaskedLM.from_pretrained("/content/drive/MyDrive/model/Compact_BioBERT/").to(device)

In [ ]:
# model_compactbiobert